<b>Dependencies</b>

In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from IPython.display import display, Image

#---

#For random # generation
import random
# Import the datetime module from the datetime library.
from datetime import datetime
# Used to find cities with citipy module from an awesome MIT dude
from citipy import citipy
# Import the requests library
import requests
# Import the API key for OWM.
from config import weather_api_key
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
#Time, formatted as today's date
today = time.strftime("%x")

#---

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress
import time
# Load in csv
#city_data_df = pd.read_csv("/Users/rommellmontenegro/Desktop/GitHub R/MyRepo/Week 6 - Lessons/Instruction/World_Weather_Analysis/weather_data/cities.csv")
#city_data_df

<b>Create a set of tuples as coordinates from random lats and longs</b>

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
coordinates

[(-80.33761423550891, 14.487346717474878),
 (-11.881020218713417, -50.72972753863607),
 (-20.859318836590134, -158.60096443043105),
 (-30.504556174747535, -114.19687578197582),
 (46.26122976158442, -139.61728659926962),
 (-24.842524486844027, -170.78322957006233),
 (-12.417128268547017, 173.44651599125558),
 (-10.845396073010036, -82.0069422957869),
 (-63.809261999596714, -112.37952148687577),
 (-6.407409966966483, -89.64045951943346),
 (0.3785257117567511, 97.353696103539),
 (75.02438478388268, 58.509110727097436),
 (50.420821632590076, 106.0983190574679),
 (-69.60629097834757, -121.98069853184404),
 (-67.58252437604406, -176.97491101194257),
 (-64.22365686229337, -7.709968948474398),
 (-13.586616701495245, 119.66947411438554),
 (29.23611120959208, -133.0993808005971),
 (16.460362666603288, -33.34130033305877),
 (23.788193668188967, -155.70982311602032),
 (-16.20387932565791, 164.13107183112197),
 (20.15279663076815, -130.11318277290655),
 (-10.508781248012284, 178.27256742421196),
 (

<b>Take coordinates, find the closest city and add that city to list of cities </b>

In [3]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

767

In [4]:
cities

['bredasdorp',
 'formoso do araguaia',
 'avarua',
 'rikitea',
 'sitka',
 'vaini',
 'sola',
 'chimbote',
 'punta arenas',
 'san cristobal',
 'sibolga',
 'amderma',
 'naushki',
 'cape town',
 'waingapu',
 'pacific grove',
 'ponta do sol',
 'kahului',
 'luganville',
 'lompoc',
 'asau',
 'bluff',
 'ambulu',
 'lolua',
 'bengkulu',
 'shakawe',
 'srednekolymsk',
 'nemuro',
 'ulaangom',
 'ilulissat',
 'yar-sale',
 'vestmannaeyjar',
 'kapaa',
 'haines junction',
 'roros',
 'amapa',
 'iqaluit',
 'bogo',
 'carnarvon',
 'lebu',
 'yellowknife',
 'trairi',
 'ubata',
 'te anau',
 'maragogi',
 'norman wells',
 'labutta',
 'kulhudhuffushi',
 'dien bien',
 'mataura',
 'touros',
 'port alfred',
 'esperance',
 'ushuaia',
 'hermanus',
 'taolanaro',
 'mount gambier',
 'hervey bay',
 'yangcun',
 'syamzha',
 'nizhneyansk',
 'cidreira',
 'kaitangata',
 'busselton',
 'katsuura',
 'jamestown',
 'constitucion',
 'faanui',
 'mar del plata',
 'mys shmidta',
 'castro',
 'fairbanks',
 'erzin',
 'pasighat',
 'zaranj',

<b> API Request in batches of 50</b>

In [6]:
import time
##### Create an empty list to hold the weather data.
city_data = []
##### Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

##### Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    #print(city_url)

    # Log the URL, record, and set numbers and the city.
    print(city_url)
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_descrip = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_descrip})
        time.sleep(.001)
#If an error is experienced, skip the city.
    except KeyError:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=bredasdorp
Processing Record 1 of Set 1 | bredasdorp
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=formoso+do+araguaia
Processing Record 2 of Set 1 | formoso do araguaia
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=avarua
Processing Record 3 of Set 1 | avarua
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=rikitea
Processing Record 4 of Set 1 | rikitea
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=sitka
Processing Record 5 of Set 1 | sitka
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=vaini
Processing Record 6 of Set 1 

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=ushuaia
Processing Record 4 of Set 2 | ushuaia
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=hermanus
Processing Record 5 of Set 2 | hermanus
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=taolanaro
Processing Record 6 of Set 2 | taolanaro
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=mount+gambier
Processing Record 7 of Set 2 | mount gambier
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=hervey+bay
Processing Record 8 of Set 2 | hervey bay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=yangcun
Processing Record 9 of Set 2 | yangcun
http://api.openweathermap.org/data/2.5/weather?u

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=tasiilaq
Processing Record 8 of Set 3 | tasiilaq
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=upernavik
Processing Record 9 of Set 3 | upernavik
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=baykit
Processing Record 10 of Set 3 | baykit
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=sindand
Processing Record 11 of Set 3 | sindand
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=newport
Processing Record 12 of Set 3 | newport
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=astipalaia
Processing Record 13 of Set 3 | astipalaia
City not found. Skipping...
http://api.openweathermap.org/

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=bajos+de+haina
Processing Record 12 of Set 4 | bajos de haina
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=anito
Processing Record 13 of Set 4 | anito
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=puerto+ayora
Processing Record 14 of Set 4 | puerto ayora
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=tarakan
Processing Record 15 of Set 4 | tarakan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=along
Processing Record 16 of Set 4 | along
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=tuktoyaktuk
Processing Record 17 of Set 4 | tuktoyaktuk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=tartagal
Processing Record 16 of Set 5 | tartagal
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=omsukchan
Processing Record 17 of Set 5 | omsukchan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=borovoy
Processing Record 18 of Set 5 | borovoy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=bambous+virieux
Processing Record 19 of Set 5 | bambous virieux
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=altensteig
Processing Record 20 of Set 5 | altensteig
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=kodiak
Processing Record 21 of Set 5 | kodiak
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=

...

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=khonuu
Processing Record 35 of Set 11 | khonuu
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=nuristan
Processing Record 36 of Set 11 | nuristan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=umm+lajj
Processing Record 37 of Set 11 | umm lajj
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=hermiston
Processing Record 38 of Set 11 | hermiston
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=igarka
Processing Record 39 of Set 11 | igarka
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=itoman
Processing Record 40 of Set 11 | itoman
http://api.openweathermap.org/data/2.5/weather?units=Imper

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=ngukurr
Processing Record 38 of Set 12 | ngukurr
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=tooele
Processing Record 39 of Set 12 | tooele
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=xining
Processing Record 40 of Set 12 | xining
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=beaverlodge
Processing Record 41 of Set 12 | beaverlodge
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=diu
Processing Record 42 of Set 12 | diu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=birjand
Processing Record 43 of Set 12 | birjand
http://api.openweathermap.org/data/2.5/weather?units=Imperial&AP

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=nuuk
Processing Record 40 of Set 13 | nuuk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=bosaso
Processing Record 41 of Set 13 | bosaso
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=lypnyazhka
Processing Record 42 of Set 13 | lypnyazhka
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=antofagasta
Processing Record 43 of Set 13 | antofagasta
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=mahibadhoo
Processing Record 44 of Set 13 | mahibadhoo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=xam+nua
Processing Record 45 of Set 13 | xam nua
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=chumikan
Processing Record 43 of Set 14 | chumikan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=gorno-chuyskiy
Processing Record 44 of Set 14 | gorno-chuyskiy
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=oga
Processing Record 45 of Set 14 | oga
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=yatou
Processing Record 46 of Set 14 | yatou
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=mildura
Processing Record 47 of Set 14 | mildura
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=tari
Processing Record 48 of Set 14 | tari
http://api.openweathermap.org/data/2.5/weather?units=Imperial&

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=kuhestan
Processing Record 45 of Set 15 | kuhestan
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=nelson+bay
Processing Record 46 of Set 15 | nelson bay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=zajecar
Processing Record 47 of Set 15 | zajecar
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=porto+walter
Processing Record 48 of Set 15 | porto walter
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=necochea
Processing Record 49 of Set 15 | necochea
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6e6aa0ff189258e8f99c4b9e61e34fca&q=romitan
Processing Record 50 of Set 15 | romitan
http://api.openweathermap.org/data/2.5/w

<b>DataFrame Creation</b>

In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
len(city_data_df)

703

In [8]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Bredasdorp,ZA,-34.53,20.04,41.00,100,76,2.24,broken clouds
1,Avarua,CK,-21.21,-159.78,75.20,88,0,20.80,clear sky
2,Rikitea,PF,-23.12,-134.97,66.22,76,100,25.19,overcast clouds
3,Sitka,US,57.05,-135.33,64.40,59,75,14.99,broken clouds
4,Vaini,TO,-21.20,-175.20,77.00,94,75,6.93,broken clouds
...,...,...,...,...,...,...,...,...,...
698,Lumphat,KH,13.49,106.98,86.85,68,66,1.16,broken clouds
699,Bulgan,MN,48.81,103.53,67.84,69,100,7.63,light rain
700,Soe,ID,-9.86,124.28,86.31,23,0,12.26,clear sky
701,Tagusao,PH,9.19,117.81,86.22,66,32,4.38,scattered clouds


In [9]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")